In [1]:
import numpy as np
import pandas as pd
import time
import random
from random import randint
from threading import Thread
from queue import Queue
from tqdm import tqdm

In [2]:
import selenium
from selenium import webdriver

import typing
from typing import List, Dict

In [3]:
from selenium.webdriver import FirefoxOptions

opts = FirefoxOptions()
opts.add_argument("--headless")
browser = webdriver.Firefox(firefox_options=opts)

C:\Users\aubin\AppData\Local\Temp/ipykernel_30432/2296700920.py:5: DeprecationWarning: use options instead of firefox_options
  browser = webdriver.Firefox(firefox_options=opts)


In [4]:
# Threading seems almost impossible...

In [5]:
import pandas as pd 

In [6]:
class ImmoWeb():
    
    def __init__(self, driver, counter = 2, debug_mode = False, start_url="https://www.immoweb.be/fr/recherche/maison/a-vendre?countries=BE&page=1&orderBy=relevance"):
        self.driver = driver
        self.start_url = start_url
        self.advert_urls = []
        self.advert_details = []
        self.all_data = []
        self.counter = counter
        
        self.debug = debug_mode
        
        
    def scrape(self):
        """
        Scrape methods calls other methods in the class in order. TODO add threading support.
        """
        
        # work in batches of 5
        urls = self.readPagination(5)

        for url in tqdm(self.advert_urls):
            data = self.readAdPage(url)
            self.all_data.append(data)

        return [self.all_data, self.counter]
            
        
    def page_advert_urls(self, page_url, url_piece = "annonce"):
        """
        Method that gathers urls for each individual ad. 
        
        :page_url: a url that links to a page containing several urls of individual ads
        :return: a list containing all the urls to the front pages.
        """
        # Init empty list to return gathered 
        url = []
        
        # call the driver get method with a time delay.
        self.driver.get(page_url)
        time.sleep(random.uniform(0.3, 0.8))
        
        # find all the links to individual ads.
        for elem in self.driver.find_elements_by_tag_name("a"):
            link = elem.get_attribute("href")
            if not link is None and url_piece in link:
                self.advert_urls.append(link)
                url.append(link)
                
        return url
       
    def readPagination(self, batches):
        """
        readPagination gathers all pages that contain independent ads. The first one is hardcoded and 
        aferwards it formats the URL as a string until no new ads are found.
        
        :return: a list of URLs that link to each page of a search query. 
        """
        # initialize a list for storing all the seperate links to individual ads.
        batch = 0
        max_batch = batches
        
        urls = ['init']
        
        # loop until no results are found
        while batch < max_batch:
            
            page_url = f"https://www.immoweb.be/fr/recherche/maison/a-vendre?countries=BE&page={self.counter}&orderBy=postal_code"
            urls = self.page_advert_urls(page_url)
            
            adverts_amount = len(urls)
            self.counter += 1
            batch += 1
            
        return urls
        
            
    def readAdPage(self, inp_url):
        """
        Method that reads all the information on the page and writes it to a dict using the key, 
        value pairs used on the website in question. 
    
        :inp_url: get the input URL that we will be gathering information from.
        :return: a dict that stores scraped data
        """

        # initialize some variables.
        detailsKeys = []
        detailsValues = []
        details = {}

        # grab the URL page code
        self.driver.get(inp_url)

        # find the tables in the page and split data in 2 types, keys and values.
        for desc_list in self.driver.find_elements_by_tag_name("th"):
            if desc_list.text != "":
                detailsKeys.append(desc_list.text)

        for desc_list in self.driver.find_elements_by_tag_name("td"):
            if desc_list.text != "":
                if desc_list.find_elements_by_tag_name("a") == []:
                    if desc_list.find_elements_by_tag_name("span") != []:
                        detail = desc_list.text.split("\n")
                        detailsValues.extend(detail[:-1])
                    else:
                        detailsValues.append(desc_list.text)

        # build a dict containing all gathered data
        try:
            for idx, x in enumerate(detailsKeys):
                details[x] = detailsValues[idx]
        except:
            pass

        self.advert_details.append(details)

        return details
            
            

In [7]:
def threadedScraper(driver, first_index):
    scraper = ImmoWeb(driver = driver, debug_mode = True, counter = first_index)
    data, data_read = scraper.scrape()
    dataframe = pd.DataFrame(data)
    filename = f"immoweb/immoweb_batch_{first_index+1}_{data_read}.csv"
    dataframe.to_csv(filename)


In [8]:
data_read = 275


In [9]:
while data_read < 330:
    driver1 = webdriver.Firefox()
    try: 
        threadedScraper(driver1, data_read)
    except:
        print(f"FAILURE AT PAGE {data_read}")
    
    driver1.quit()
    data_read += 5


100%|██████████| 150/150 [06:26<00:00,  2.58s/it]


In [ ]:
data_read = 0

while data_read < 200:
    driver1 = webdriver.Firefox()
    try: 
        threadedScraper(driver1, data_read)
    except:
        print(f"FAILURE AT PAGE {data_read}")
    
    driver1.quit()
    data_read += 5

 95%|█████████▌| 146/153 [08:10<00:16,  2.42s/it]